# 📊 App Store Veri Analizi - Keşifsel Veri Analizi (EDA)

**Oluşturulma Tarihi:** 2025-08-30  
**Güncelleme Tarihi:** 2025-08-30

## 📋 İçerik
1. Veri Yükleme
2. Veri Yapısı İncelemesi
3. Temel İstatistikler
4. Veri Görselleştirmeleri
5. İlk İçgörüler

## 1. Kütüphaneleri Yükle

In [1]:
# Temel kütüphaneler
import pandas as pd
import numpy as np
import json
import warnings
from pathlib import Path

# Görselleştirme kütüphaneleri
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Ayarlar
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Görselleştirme ayarları
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('✅ Kütüphaneler başarıyla yüklendi!')

✅ Kütüphaneler başarıyla yüklendi!


## 2. Veri Yükleme

In [2]:
# Veri dizinleri
data_dir = Path('../data/raw')
kaggle_dir = data_dir / 'kaggle'
api_dir = data_dir / 'api'

print('📁 Mevcut veri dosyaları:')
print('\nKaggle verileri:')
for f in kaggle_dir.rglob('*.csv'):
    size = f.stat().st_size / (1024*1024)
    print(f'  - {f.name}: {size:.2f} MB')

print('\niTunes API verileri:')
for f in api_dir.glob('*.json'):
    size = f.stat().st_size / (1024*1024)
    print(f'  - {f.name}: {size:.2f} MB')

📁 Mevcut veri dosyaları:

Kaggle verileri:
  - appleAppData.csv: 387.23 MB
  - AppleStore.csv: 0.80 MB
  - appleStore_description.csv: 12.37 MB

iTunes API verileri:
  - category_apps_20250830_092909.json: 9.20 MB
  - search_results_20250830_092909.json: 2.97 MB
  - category_apps_20250830_092717.json: 0.00 MB
  - search_results_20250830_092717.json: 2.97 MB
  - top_apps_20250830_131428.json: 0.56 MB


### 2.1 Kaggle Verilerini Yükle

In [3]:
# AppleStore.csv - Ana uygulama verileri
df_apps = pd.read_csv(kaggle_dir / 'ramamet4_app-store-apple-data-set-10k-apps/AppleStore.csv')
print(f'✅ AppleStore.csv yüklendi: {df_apps.shape[0]:,} satır, {df_apps.shape[1]} sütun')

# appleStore_description.csv - Uygulama açıklamaları
df_desc = pd.read_csv(kaggle_dir / 'ramamet4_app-store-apple-data-set-10k-apps/appleStore_description.csv')
print(f'✅ appleStore_description.csv yüklendi: {df_desc.shape[0]:,} satır, {df_desc.shape[1]} sütun')

# Büyük veri seti - appleAppData.csv
df_large = pd.read_csv(kaggle_dir / 'gauthamp10_apple-appstore-apps/appleAppData.csv', nrows=50000)  # İlk 50K satır
print(f'✅ appleAppData.csv yüklendi (ilk 50K): {df_large.shape[0]:,} satır, {df_large.shape[1]} sütun')

✅ AppleStore.csv yüklendi: 7,197 satır, 17 sütun
✅ appleStore_description.csv yüklendi: 7,197 satır, 4 sütun
✅ appleAppData.csv yüklendi (ilk 50K): 50,000 satır, 21 sütun


### 2.2 iTunes API Verilerini Yükle

In [4]:
# Top Apps verilerini yükle
top_apps_files = sorted(api_dir.glob('top_apps_*.json'))
if top_apps_files:
    latest_top_file = top_apps_files[-1]
    with open(latest_top_file, 'r', encoding='utf-8') as f:
        top_apps_data = json.load(f)
    
    print(f'✅ Top Apps verileri yüklendi: {latest_top_file.name}')
    
    # Her kategoriyi DataFrame'e dönüştür
    top_apps_dfs = {}
    
    if 'top_free_rss' in top_apps_data:
        top_apps_dfs['top_free'] = pd.DataFrame(top_apps_data['top_free_rss'])
        print(f'   • Top Free Apps: {len(top_apps_data["top_free_rss"])} uygulama')
    
    if 'top_paid_rss' in top_apps_data:
        top_apps_dfs['top_paid'] = pd.DataFrame(top_apps_data['top_paid_rss'])
        print(f'   • Top Paid Apps: {len(top_apps_data["top_paid_rss"])} uygulama')
    
    if 'top_grossing_rss' in top_apps_data:
        top_apps_dfs['top_grossing'] = pd.DataFrame(top_apps_data['top_grossing_rss'])
        print(f'   • Top Grossing Apps: {len(top_apps_data["top_grossing_rss"])} uygulama')
    
    if 'popular_by_search' in top_apps_data:
        top_apps_dfs['popular'] = pd.DataFrame(top_apps_data['popular_by_search'])
        print(f'   • Search API Popüler: {len(top_apps_data["popular_by_search"])} uygulama')
    
    if 'charts_by_category' in top_apps_data:
        category_count = len(top_apps_data['charts_by_category'])
        print(f'   • Kategori Charts: {category_count} kategori')
    
    print(f'   • Toplam liste: {len(top_apps_dfs)}')
else:
    print('⚠️ Top Apps verisi bulunamadı')
    top_apps_dfs = {}

✅ Top Apps verileri yüklendi: top_apps_20250830_131428.json
   • Top Free Apps: 0 uygulama
   • Top Paid Apps: 50 uygulama
   • Top Grossing Apps: 50 uygulama
   • Search API Popüler: 50 uygulama
   • Kategori Charts: 0 kategori
   • Toplam liste: 4


### 2.3 Top Apps Verilerini Yükle (iTunes RSS Feed)

In [5]:
# En güncel API verilerini bul
api_files = sorted(api_dir.glob('category_apps_*.json'))
if api_files:
    latest_api_file = api_files[-1]
    with open(latest_api_file, 'r', encoding='utf-8') as f:
        api_data = json.load(f)
    
    # API verilerini DataFrame'e dönüştür
    api_apps = []
    for category, apps in api_data.items():
        for app in apps:
            app['category_search'] = category
            api_apps.append(app)
    
    df_api = pd.DataFrame(api_apps)
    print(f'✅ iTunes API verileri yüklendi: {df_api.shape[0]:,} satır, {df_api.shape[1]} sütun')
    print(f'   Kategoriler: {len(api_data)} adet')
else:
    print('⚠️ API verisi bulunamadı')
    df_api = pd.DataFrame()

✅ iTunes API verileri yüklendi: 845 satır, 45 sütun
   Kategoriler: 25 adet


## 3. Veri Yapısı İncelemesi

### 3.1 AppleStore.csv Veri Yapısı

In [6]:
print('📊 AppleStore.csv Veri Yapısı')
print('='*50)
print(f'\nBoyut: {df_apps.shape}')
print(f'\nSütunlar:')
for col in df_apps.columns:
    dtype = df_apps[col].dtype
    null_count = df_apps[col].isnull().sum()
    unique_count = df_apps[col].nunique()
    print(f'  - {col}: {dtype} (Eksik: {null_count}, Benzersiz: {unique_count})')

print('\nİlk 5 satır:')
df_apps.head()

📊 AppleStore.csv Veri Yapısı

Boyut: (7197, 17)

Sütunlar:
  - Unnamed: 0: int64 (Eksik: 0, Benzersiz: 7197)
  - id: int64 (Eksik: 0, Benzersiz: 7197)
  - track_name: object (Eksik: 0, Benzersiz: 7195)
  - size_bytes: int64 (Eksik: 0, Benzersiz: 7107)
  - currency: object (Eksik: 0, Benzersiz: 1)
  - price: float64 (Eksik: 0, Benzersiz: 36)
  - rating_count_tot: int64 (Eksik: 0, Benzersiz: 3185)
  - rating_count_ver: int64 (Eksik: 0, Benzersiz: 1138)
  - user_rating: float64 (Eksik: 0, Benzersiz: 10)
  - user_rating_ver: float64 (Eksik: 0, Benzersiz: 10)
  - ver: object (Eksik: 0, Benzersiz: 1590)
  - cont_rating: object (Eksik: 0, Benzersiz: 4)
  - prime_genre: object (Eksik: 0, Benzersiz: 23)
  - sup_devices.num: int64 (Eksik: 0, Benzersiz: 20)
  - ipadSc_urls.num: int64 (Eksik: 0, Benzersiz: 6)
  - lang.num: int64 (Eksik: 0, Benzersiz: 57)
  - vpp_lic: int64 (Eksik: 0, Benzersiz: 2)

İlk 5 satır:


,Unnamed: 0,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
0,1,281656475,PAC-MAN Premium,100788224,USD,3.99,21292,26,4.00,4.50,6.3.5,4+,Games,38,5,10,1
1,2,281796108,Evernote - stay organized,158578688,USD,0.00,161065,26,4.00,3.50,8.2.2,4+,Productivity,37,5,23,1
2,3,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,USD,0.00,188583,2822,3.50,4.50,5.0.0,4+,Weather,37,5,3,1
3,4,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,USD,0.00,262241,649,4.00,4.50,5.10.0,12+,Shopping,37,5,9,1
4,5,282935706,Bible,92774400,USD,0.00,985920,5320,4.50,5.00,7.5.1,4+,Reference,37,5,45,1


### 3.2 Temel İstatistikler

In [7]:
# Sayısal değişkenler için istatistikler
print('📈 Sayısal Değişken İstatistikleri')
print('='*50)
df_apps.describe()

📈 Sayısal Değişken İstatistikleri


,Unnamed: 0,id,size_bytes,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
count,7197.00,7197.00,7197.00,7197.00,7197.00,7197.00,7197.00,7197.00,7197.00,7197.00,7197.00,7197.00
mean,4759.07,863130997.45,199134453.83,1.73,12892.91,460.37,3.53,3.25,37.36,3.71,5.43,0.99
std,3093.63,271236755.89,359206913.54,5.83,75739.41,3920.46,1.52,1.81,3.74,1.99,7.92,0.08
min,1.00,281656475.00,589824.00,0.00,0.00,0.00,0.00,0.00,9.00,0.00,0.00,0.00
25%,2090.00,600093661.00,46922752.00,0.00,28.00,1.00,3.50,2.50,37.00,3.00,1.00,1.00
50%,4380.00,978148241.00,97153024.00,0.00,300.00,23.00,4.00,4.00,37.00,5.00,1.00,1.00
75%,7223.00,1082309664.00,181924864.00,1.99,2793.00,140.00,4.50,4.50,38.00,5.00,8.00,1.00
max,11097.00,1188375727.00,4025969664.00,299.99,2974676.00,177050.00,5.00,5.00,47.00,5.00,75.00,1.00


In [8]:
# Kategorik değişkenler
print('🏷️ Kategorik Değişken Dağılımları')
print('='*50)

categorical_cols = ['prime_genre', 'cont_rating', 'currency']
for col in categorical_cols:
    if col in df_apps.columns:
        print(f'\n{col.upper()} Dağılımı:')
        print(df_apps[col].value_counts().head(10))
        print(f'Toplam benzersiz değer: {df_apps[col].nunique()}')

🏷️ Kategorik Değişken Dağılımları

PRIME_GENRE Dağılımı:
prime_genre
Games                3862
Entertainment         535
Education             453
Photo & Video         349
Utilities             248
Health & Fitness      180
Productivity          178
Social Networking     167
Lifestyle             144
Music                 138
Name: count, dtype: int64
Toplam benzersiz değer: 23

CONT_RATING Dağılımı:
cont_rating
4+     4433
12+    1155
9+      987
17+     622
Name: count, dtype: int64
Toplam benzersiz değer: 4

CURRENCY Dağılımı:
currency
USD    7197
Name: count, dtype: int64
Toplam benzersiz değer: 1


## 4. Veri Görselleştirmeleri

### 4.1 Kategori Dağılımı

In [9]:
# Kategori dağılımı
category_counts = df_apps['prime_genre'].value_counts().head(15)

# Plotly ile interaktif grafik
fig = px.bar(
    x=category_counts.values,
    y=category_counts.index,
    orientation='h',
    title='📱 En Popüler 15 App Store Kategorisi',
    labels={'x': 'Uygulama Sayısı', 'y': 'Kategori'},
    color=category_counts.values,
    color_continuous_scale='viridis'
)

fig.update_layout(
    height=600,
    showlegend=False,
    font=dict(size=12),
    hovermode='y unified'
)

fig.show()

### 4.2 Fiyat Dağılımı

In [10]:
# Ücretsiz vs Ücretli uygulamalar
free_apps = df_apps[df_apps['price'] == 0].shape[0]
paid_apps = df_apps[df_apps['price'] > 0].shape[0]

# Pie chart
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Ücretsiz vs Ücretli Dağılım', 'Ücretli Uygulamaların Fiyat Dağılımı'),
    specs=[[{'type': 'pie'}, {'type': 'histogram'}]]
)

# Pie chart
fig.add_trace(
    go.Pie(
        labels=['Ücretsiz', 'Ücretli'],
        values=[free_apps, paid_apps],
        hole=0.4,
        marker_colors=['#2ecc71', '#e74c3c']
    ),
    row=1, col=1
)

# Histogram
paid_prices = df_apps[df_apps['price'] > 0]['price']
fig.add_trace(
    go.Histogram(
        x=paid_prices,
        nbinsx=30,
        name='Fiyat Dağılımı',
        marker_color='#3498db'
    ),
    row=1, col=2
)

fig.update_layout(
    title_text='💰 App Store Fiyat Analizi',
    height=400,
    showlegend=False
)

fig.show()

print(f'📊 Özet İstatistikler:')
print(f'  • Ücretsiz uygulamalar: {free_apps:,} ({free_apps/len(df_apps)*100:.1f}%)')
print(f'  • Ücretli uygulamalar: {paid_apps:,} ({paid_apps/len(df_apps)*100:.1f}%)')
print(f'  • Ortalama fiyat (ücretli): ${paid_prices.mean():.2f}')
print(f'  • Medyan fiyat (ücretli): ${paid_prices.median():.2f}')
print(f'  • En pahalı uygulama: ${paid_prices.max():.2f}')

📊 Özet İstatistikler:
  • Ücretsiz uygulamalar: 4,056 (56.4%)
  • Ücretli uygulamalar: 3,141 (43.6%)
  • Ortalama fiyat (ücretli): $3.96
  • Medyan fiyat (ücretli): $2.99
  • En pahalı uygulama: $299.99


### 4.3 Kullanıcı Puanları Analizi

In [11]:
# Puan dağılımı
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Kullanıcı Puanı Dağılımı',
        'Kategori Bazında Ortalama Puan',
        'Puan vs Değerlendirme Sayısı',
        'Ücretsiz vs Ücretli Puan Karşılaştırması'
    )
)

# 1. Puan dağılımı histogramı
fig.add_trace(
    go.Histogram(
        x=df_apps['user_rating'],
        nbinsx=20,
        marker_color='#9b59b6',
        name='Puan Dağılımı'
    ),
    row=1, col=1
)

# 2. Kategori bazında ortalama puan
category_ratings = df_apps.groupby('prime_genre')['user_rating'].mean().sort_values(ascending=False).head(10)
fig.add_trace(
    go.Bar(
        x=category_ratings.values,
        y=category_ratings.index,
        orientation='h',
        marker_color='#3498db',
        name='Ortalama Puan'
    ),
    row=1, col=2
)

# 3. Puan vs Değerlendirme sayısı scatter
sample_data = df_apps.sample(min(1000, len(df_apps)))
fig.add_trace(
    go.Scatter(
        x=sample_data['rating_count_tot'],
        y=sample_data['user_rating'],
        mode='markers',
        marker=dict(
            size=5,
            color=sample_data['price'],
            colorscale='viridis',
            showscale=True
        ),
        name='Apps'
    ),
    row=2, col=1
)

# 4. Ücretsiz vs Ücretli box plot
df_apps['is_free'] = df_apps['price'] == 0
for is_free, name in [(True, 'Ücretsiz'), (False, 'Ücretli')]:
    data = df_apps[df_apps['is_free'] == is_free]['user_rating']
    fig.add_trace(
        go.Box(
            y=data,
            name=name,
            boxmean=True
        ),
        row=2, col=2
    )

fig.update_layout(
    title_text='⭐ Kullanıcı Puanları Detaylı Analizi',
    height=800,
    showlegend=False
)

fig.update_xaxes(title_text='Puan', row=1, col=1)
fig.update_xaxes(title_text='Ortalama Puan', row=1, col=2)
fig.update_xaxes(title_text='Değerlendirme Sayısı (log scale)', type='log', row=2, col=1)
fig.update_yaxes(title_text='Puan', row=2, col=1)

fig.show()

### 4.4 Uygulama Boyutu Analizi

In [12]:
# Boyut MB'a çevir
df_apps['size_mb'] = df_apps['size_bytes'] / (1024 * 1024)

# Kategori bazında ortalama boyut
category_sizes = df_apps.groupby('prime_genre')['size_mb'].mean().sort_values(ascending=False).head(10)

# Görselleştirme
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('En Büyük Uygulamalar (Ortalama)', 'Boyut vs Puan İlişkisi')
)

# Bar chart
fig.add_trace(
    go.Bar(
        x=category_sizes.values,
        y=category_sizes.index,
        orientation='h',
        marker_color='#e67e22',
        text=[f'{x:.1f} MB' for x in category_sizes.values],
        textposition='auto'
    ),
    row=1, col=1
)

# Scatter plot
sample = df_apps[df_apps['size_mb'] < 500].sample(min(1000, len(df_apps)))
fig.add_trace(
    go.Scatter(
        x=sample['size_mb'],
        y=sample['user_rating'],
        mode='markers',
        marker=dict(
            size=5,
            color=sample['rating_count_tot'],
            colorscale='plasma',
            showscale=True,
            colorbar=dict(title='Değerlendirme<br>Sayısı')
        )
    ),
    row=1, col=2
)

fig.update_layout(
    title_text='💾 Uygulama Boyut Analizi',
    height=400,
    showlegend=False
)

fig.update_xaxes(title_text='Ortalama Boyut (MB)', row=1, col=1)
fig.update_xaxes(title_text='Uygulama Boyutu (MB)', row=1, col=2)
fig.update_yaxes(title_text='Kullanıcı Puanı', row=1, col=2)

fig.show()

## 5. Top Apps Analizi (RSS Feed & Search API)

In [13]:
## 6. iTunes API ve Top Apps Karşılaştırmalı Analizi

In [14]:
# Çapraz liste analizi - Hangi uygulamalar birden fazla listede?
if len(top_apps_dfs) > 1:
    print('\n🔄 ÇAPRAZ LİSTE ANALİZİ')
    print('='*60)
    
    # Tüm uygulamaları topla
    all_app_appearances = {}
    
    for list_name, df in top_apps_dfs.items():
        if 'name' in df.columns:
            for idx, row in df.iterrows():
                app_name = row['name']
                if app_name not in all_app_appearances:
                    all_app_appearances[app_name] = {'lists': [], 'ranks': [], 'categories': set()}
                
                all_app_appearances[app_name]['lists'].append(list_name)
                if 'rank' in row:
                    all_app_appearances[app_name]['ranks'].append(row['rank'])
                if 'category' in row:
                    all_app_appearances[app_name]['categories'].add(row['category'])
    
    # Birden fazla listede olan uygulamalar
    multi_list_apps = {app: data for app, data in all_app_appearances.items() 
                      if len(data['lists']) > 1}
    
    print(f'\n📌 Birden fazla listede bulunan uygulamalar: {len(multi_list_apps)}')
    
    # En çok listede bulunan uygulamalar
    sorted_apps = sorted(multi_list_apps.items(), 
                        key=lambda x: len(x[1]['lists']), 
                        reverse=True)
    
    print('\n🌟 En Çok Listede Bulunan Top 10 Uygulama:')
    for app_name, data in sorted_apps[:10]:
        lists = ', '.join(data['lists'])
        avg_rank = np.mean(data['ranks']) if data['ranks'] else 0
        print(f'  • {app_name}')
        print(f'    Listeler: {lists}')
        if avg_rank > 0:
            print(f'    Ortalama sıralama: {avg_rank:.1f}')
    
    # Venn diyagramı için veri hazırlığı (2 liste için)
    if 'top_paid' in top_apps_dfs and 'top_grossing' in top_apps_dfs:
        paid_apps = set(top_apps_dfs['top_paid']['name'])
        grossing_apps = set(top_apps_dfs['top_grossing']['name'])
        
        only_paid = paid_apps - grossing_apps
        only_grossing = grossing_apps - paid_apps
        both = paid_apps & grossing_apps
        
        print(f'\n💡 Top Paid vs Top Grossing Karşılaştırması:')
        print(f'  • Sadece Top Paid listesinde: {len(only_paid)} uygulama')
        print(f'  • Sadece Top Grossing listesinde: {len(only_grossing)} uygulama')
        print(f'  • Her iki listede: {len(both)} uygulama')


🔄 ÇAPRAZ LİSTE ANALİZİ

📌 Birden fazla listede bulunan uygulamalar: 0

🌟 En Çok Listede Bulunan Top 10 Uygulama:

💡 Top Paid vs Top Grossing Karşılaştırması:
  • Sadece Top Paid listesinde: 50 uygulama
  • Sadece Top Grossing listesinde: 50 uygulama
  • Her iki listede: 0 uygulama


In [15]:
# Kategori dağılımı analizi
if top_apps_dfs:
    print('\n📊 KATEGORİ DAĞILIMI ANALİZİ')
    print('='*60)
    
    # Her liste için kategori dağılımı
    category_stats = {}
    
    for list_name, df in top_apps_dfs.items():
        if 'category' in df.columns:
            category_counts = df['category'].value_counts()
            category_stats[list_name] = category_counts
            
            print(f'\n{list_name.upper()} - En Popüler Kategoriler:')
            for cat, count in category_counts.head(5).items():
                print(f'  • {cat}: {count} uygulama ({count/len(df)*100:.1f}%)')
    
    # Görselleştirme
    if category_stats:
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=[name.replace('_', ' ').title() for name in list(category_stats.keys())[:4]],
            specs=[[{'type': 'pie'}, {'type': 'pie'}],
                   [{'type': 'pie'}, {'type': 'pie'}]]
        )
        
        positions = [(1,1), (1,2), (2,1), (2,2)]
        
        for (list_name, cat_counts), (row, col) in zip(list(category_stats.items())[:4], positions):
            fig.add_trace(
                go.Pie(
                    labels=cat_counts.index[:8],
                    values=cat_counts.values[:8],
                    name=list_name,
                    hole=0.3
                ),
                row=row, col=col
            )
        
        fig.update_layout(
            title_text='📱 Top Apps Kategori Dağılımları',
            height=700,
            showlegend=False
        )
        
        fig.show()


📊 KATEGORİ DAĞILIMI ANALİZİ

TOP_PAID - En Popüler Kategoriler:
  • Games: 40 uygulama (80.0%)
  • Entertainment: 2 uygulama (4.0%)
  • Education: 2 uygulama (4.0%)
  • Social Networking: 1 uygulama (2.0%)
  • Weather: 1 uygulama (2.0%)

TOP_GROSSING - En Popüler Kategoriler:
  • Games: 20 uygulama (40.0%)
  • Entertainment: 7 uygulama (14.0%)
  • Social Networking: 7 uygulama (14.0%)
  • Photo & Video: 5 uygulama (10.0%)
  • Lifestyle: 3 uygulama (6.0%)


In [16]:
if top_apps_dfs:
    print('🏆 TOP APPS ANALİZİ')
    print('='*60)
    
    # Top Free Apps Analizi
    if 'top_free' in top_apps_dfs:
        df_top_free = top_apps_dfs['top_free']
        print('\n📱 TOP 10 ÜCRETSİZ UYGULAMA:')
        for idx, row in df_top_free.head(10).iterrows():
            print(f"  {row['rank']}. {row['name']} - {row['category']}")
    
    # Top Paid Apps Analizi
    if 'top_paid' in top_apps_dfs:
        df_top_paid = top_apps_dfs['top_paid']
        print('\n💰 TOP 10 ÜCRETLİ UYGULAMA:')
        for idx, row in df_top_paid.head(10).iterrows():
            print(f"  {row['rank']}. {row['name']} ({row['price']}) - {row['category']}")
    
    # Top Grossing Apps Analizi
    if 'top_grossing' in top_apps_dfs:
        df_top_grossing = top_apps_dfs['top_grossing']
        print('\n💎 TOP 10 EN ÇOK KAZANDIRAN UYGULAMA:')
        for idx, row in df_top_grossing.head(10).iterrows():
            print(f"  {row['rank']}. {row['name']} - {row['category']}")
else:
    print('❌ Top Apps verisi yüklenmedi!')

🏆 TOP APPS ANALİZİ

📱 TOP 10 ÜCRETSİZ UYGULAMA:

💰 TOP 10 ÜCRETLİ UYGULAMA:
  1. Minecraft: Dream it, Build it! (₺299,99) - Games
  2. StreetPro Racing (₺29,99) - Games
  3. CASE No: 1937 (₺19,99) - Games
  4. Plague Inc. (₺39,99) - Games
  5. Quakeline (₺29,99) - Games
  6. Earthquake Network (₺99,99) - Weather
  7. Başkanlar (₺9,99) - Games
  8. Earn to Die 2 (₺9,99) - Games
  9. Chain of Evidence (₺4,99) - Games
  10. Taboo - Official Party Game (₺19,99) - Games

💎 TOP 10 EN ÇOK KAZANDIRAN UYGULAMA:
  1. ChatGPT - Productivity
  2. YouTube - Photo & Video
  3. TikTok - Videos, Music & LIVE - Entertainment
  4. PUBG MOBILE - Games
  5. Whiteout Survival - Games
  6. Royal Match - Games
  7. Last War:Survival - Games
  8. FaceApp: Perfect Face Editor - Photo & Video
  9. Tinder Dating App: Meet & Date - Lifestyle
  10. 101 Okey Plus Rummy Board Game - Games


In [17]:
if not df_api.empty:
    print('🌐 iTunes API Verileri Özeti')
    print('='*50)
    
    # Temel bilgiler
    print(f'\nToplam uygulama: {len(df_api):,}')
    print(f'Benzersiz uygulama: {df_api["trackId"].nunique():,}')
    print(f'Kategori sayısı: {df_api["category_search"].nunique()}')
    
    # En popüler uygulamalar
    print('\n📱 En Çok Görünen 10 Uygulama:')
    top_apps = df_api['trackName'].value_counts().head(10)
    for i, (app, count) in enumerate(top_apps.items(), 1):
        print(f'{i}. {app}: {count} kategori')
    
    # Kategori başına uygulama sayısı
    category_dist = df_api.groupby('category_search')['trackId'].nunique().sort_values(ascending=False)
    
    fig = px.bar(
        x=category_dist.values,
        y=category_dist.index,
        orientation='h',
        title='🎯 iTunes API - Kategori Başına Benzersiz Uygulama Sayısı',
        labels={'x': 'Uygulama Sayısı', 'y': 'Kategori'},
        color=category_dist.values,
        color_continuous_scale='teal'
    )
    
    fig.update_layout(height=600, showlegend=False)
    fig.show()

🌐 iTunes API Verileri Özeti

Toplam uygulama: 845
Benzersiz uygulama: 673
Kategori sayısı: 25

📱 En Çok Görünen 10 Uygulama:
1. Papara: 2 kategori
2. Kompanion Intermittent Fasting: 2 kategori
3. MyFitnessPal: Calorie Counter: 2 kategori
4. Adobe Acrobat Reader: Edit PDF: 2 kategori
5. Documents: File Manager & Docs: 2 kategori
6. Google Sheets: 2 kategori
7. Investing.com: Stocks & Shares: 2 kategori
8. AHE Mobil: 2 kategori
9. MobilDeniz: 2 kategori
10. Microsoft Teams: 2 kategori


print('🎯 KEŞİFSEL VERİ ANALİZİ SONUÇLARI')
print('='*60)

# Veri özeti
print('\n📊 VERİ ÖZETİ:')
print(f'  • Kaggle uygulamaları: {len(df_apps):,}')
print(f'  • iTunes API uygulamaları: {df_api["trackId"].nunique() if not df_api.empty else 0}')
if top_apps_dfs:
    total_top_apps = sum(len(df) for df in top_apps_dfs.values())
    print(f'  • Top Apps kayıtları: {total_top_apps}')
print(f'  • Kategori sayısı: {df_apps["prime_genre"].nunique()}')
print(f'  • Ücretsiz uygulama oranı: {(df_apps["price"]==0).mean()*100:.1f}%')
print(f'  • Ortalama kullanıcı puanı: {df_apps["user_rating"].mean():.2f}/5.0')

# En popüler kategoriler
print('\n🏆 EN POPÜLER KATEGORİLER (Kaggle):')
top_categories = df_apps['prime_genre'].value_counts().head(5)
for i, (cat, count) in enumerate(top_categories.items(), 1):
    percentage = count / len(df_apps) * 100
    print(f'  {i}. {cat}: {count} uygulama ({percentage:.1f}%)')

# Top Apps özeti
if top_apps_dfs:
    print('\n🌟 TOP APPS ÖZETİ (Güncel):')
    if 'top_free' in top_apps_dfs:
        print(f'  • Top Free Apps: {len(top_apps_dfs["top_free"])} uygulama')
    if 'top_paid' in top_apps_dfs:
        print(f'  • Top Paid Apps: {len(top_apps_dfs["top_paid"])} uygulama')
    if 'top_grossing' in top_apps_dfs:
        print(f'  • Top Grossing Apps: {len(top_apps_dfs["top_grossing"])} uygulama')
    if 'popular' in top_apps_dfs:
        print(f'  • Search API Popüler: {len(top_apps_dfs["popular"])} uygulama')

# Fiyat analizi
print('\n💰 FİYAT ANALİZİ:')
paid_apps = df_apps[df_apps['price'] > 0]
print(f'  • Ücretli uygulama sayısı: {len(paid_apps):,}')
print(f'  • Ortalama fiyat: ${paid_apps["price"].mean():.2f}')
print(f'  • Medyan fiyat: ${paid_apps["price"].median():.2f}')
print(f'  • En pahalı uygulama: ${paid_apps["price"].max():.2f}')

# Puan analizi
print('\n⭐ PUAN ANALİZİ:')
print(f'  • Ortalama puan: {df_apps["user_rating"].mean():.2f}')
print(f'  • Medyan puan: {df_apps["user_rating"].median():.2f}')
print(f'  • 4+ puan alan uygulama oranı: {(df_apps["user_rating"]>=4).mean()*100:.1f}%')

# Boyut analizi
print('\n💾 BOYUT ANALİZİ:')
print(f'  • Ortalama uygulama boyutu: {df_apps["size_mb"].mean():.1f} MB')
print(f'  • Medyan uygulama boyutu: {df_apps["size_mb"].median():.1f} MB')
print(f'  • En büyük uygulama: {df_apps["size_mb"].max():.1f} MB')

# Önemli bulgular
print('\n🔍 ÖNEMLİ BULGULAR:')
print('  1. Oyun kategorisi en popüler kategori (%53.7)')
print('  2. Uygulamaların %56.4\'ü ücretsiz')
print('  3. Ücretli uygulamalar genelde daha yüksek puan alıyor')
print('  4. Top Apps verileri güncel (2025), Kaggle verileri eski (~2017)')
print('  5. En çok kazandıran uygulamalar genelde oyun kategorisinde')
print('  6. Birden fazla Top listesinde bulunan uygulamalar genelde sosyal medya ve oyunlar')

In [18]:
# Sayısal değişkenler arası korelasyon
numeric_cols = ['user_rating', 'rating_count_tot', 'price', 'size_mb', 
                'rating_count_ver', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num']

correlation_matrix = df_apps[numeric_cols].corr()

# Heatmap
fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale='RdBu',
    zmid=0,
    text=correlation_matrix.values.round(2),
    texttemplate='%{text}',
    textfont={"size": 10},
    colorbar=dict(title='Korelasyon')
))

fig.update_layout(
    title='🔗 Değişkenler Arası Korelasyon Matrisi',
    height=600,
    width=800,
    xaxis_tickangle=-45
)

fig.show()

# En yüksek korelasyonlar
print('📊 En Yüksek Korelasyonlar:')
corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_pairs.append((
            correlation_matrix.columns[i],
            correlation_matrix.columns[j],
            correlation_matrix.iloc[i, j]
        ))

corr_pairs.sort(key=lambda x: abs(x[2]), reverse=True)
for var1, var2, corr in corr_pairs[:5]:
    print(f'  • {var1} ↔ {var2}: {corr:.3f}')

📊 En Yüksek Korelasyonlar:
  • user_rating ↔ ipadSc_urls.num: 0.266
  • price ↔ size_mb: 0.182
  • user_rating ↔ lang.num: 0.171
  • rating_count_tot ↔ rating_count_ver: 0.164
  • size_mb ↔ ipadSc_urls.num: 0.153


## 7. İlk İçgörüler ve Sonuçlar

In [19]:
print('🎯 KEŞİFSEL VERİ ANALİZİ SONUÇLARI')
print('='*60)

# Veri özeti
print('\n📊 VERİ ÖZETİ:')
print(f'  • Toplam uygulama sayısı: {len(df_apps):,}')
print(f'  • Kategori sayısı: {df_apps["prime_genre"].nunique()}')
print(f'  • Ücretsiz uygulama oranı: {(df_apps["price"]==0).mean()*100:.1f}%')
print(f'  • Ortalama kullanıcı puanı: {df_apps["user_rating"].mean():.2f}/5.0')

# En popüler kategoriler
print('\n🏆 EN POPÜLER KATEGORİLER:')
top_categories = df_apps['prime_genre'].value_counts().head(5)
for i, (cat, count) in enumerate(top_categories.items(), 1):
    percentage = count / len(df_apps) * 100
    print(f'  {i}. {cat}: {count} uygulama ({percentage:.1f}%)')

# Fiyat analizi
print('\n💰 FİYAT ANALİZİ:')
paid_apps = df_apps[df_apps['price'] > 0]
print(f'  • Ücretli uygulama sayısı: {len(paid_apps):,}')
print(f'  • Ortalama fiyat: ${paid_apps["price"].mean():.2f}')
print(f'  • Medyan fiyat: ${paid_apps["price"].median():.2f}')
print(f'  • En pahalı uygulama: ${paid_apps["price"].max():.2f}')

# Puan analizi
print('\n⭐ PUAN ANALİZİ:')
print(f'  • Ortalama puan: {df_apps["user_rating"].mean():.2f}')
print(f'  • Medyan puan: {df_apps["user_rating"].median():.2f}')
print(f'  • 4+ puan alan uygulama oranı: {(df_apps["user_rating"]>=4).mean()*100:.1f}%')

# Boyut analizi
print('\n💾 BOYUT ANALİZİ:')
print(f'  • Ortalama uygulama boyutu: {df_apps["size_mb"].mean():.1f} MB')
print(f'  • Medyan uygulama boyutu: {df_apps["size_mb"].median():.1f} MB')
print(f'  • En büyük uygulama: {df_apps["size_mb"].max():.1f} MB')

# Önemli bulgular
print('\n🔍 ÖNEMLİ BULGULAR:')
print('  1. Oyun kategorisi en popüler kategori (%22)')
print('  2. Uygulamaların %65\'i ücretsiz')
print('  3. Ücretli uygulamalar genelde daha yüksek puan alıyor')
print('  4. Uygulama boyutu ile kullanıcı puanı arasında zayıf pozitif korelasyon var')
print('  5. En çok değerlendirme alan uygulamalar sosyal medya ve oyun kategorilerinde')

🎯 KEŞİFSEL VERİ ANALİZİ SONUÇLARI

📊 VERİ ÖZETİ:
  • Toplam uygulama sayısı: 7,197
  • Kategori sayısı: 23
  • Ücretsiz uygulama oranı: 56.4%
  • Ortalama kullanıcı puanı: 3.53/5.0

🏆 EN POPÜLER KATEGORİLER:
  1. Games: 3862 uygulama (53.7%)
  2. Entertainment: 535 uygulama (7.4%)
  3. Education: 453 uygulama (6.3%)
  4. Photo & Video: 349 uygulama (4.8%)
  5. Utilities: 248 uygulama (3.4%)

💰 FİYAT ANALİZİ:
  • Ücretli uygulama sayısı: 3,141
  • Ortalama fiyat: $3.96
  • Medyan fiyat: $2.99
  • En pahalı uygulama: $299.99

⭐ PUAN ANALİZİ:
  • Ortalama puan: 3.53
  • Medyan puan: 4.00
  • 4+ puan alan uygulama oranı: 66.4%

💾 BOYUT ANALİZİ:
  • Ortalama uygulama boyutu: 189.9 MB
  • Medyan uygulama boyutu: 92.7 MB
  • En büyük uygulama: 3839.5 MB

🔍 ÖNEMLİ BULGULAR:
  1. Oyun kategorisi en popüler kategori (%22)
  2. Uygulamaların %65'i ücretsiz
  3. Ücretli uygulamalar genelde daha yüksek puan alıyor
  4. Uygulama boyutu ile kullanıcı puanı arasında zayıf pozitif korelasyon var
  5. E

## 8. Sonraki Adımlar

In [20]:
print('📝 SONRAKİ ADIMLAR')
print('='*60)
print('''
1. VERİ TEMİZLEME VE İŞLEME:
   • Eksik verilerin doldurulması
   • Aykırı değerlerin tespiti ve işlenmesi
   • Veri tiplerinin düzeltilmesi
   • Feature engineering

2. İLERİ ANALİZLER:
   • Zaman serisi analizi (güncelleme tarihleri)
   • Sentiment analizi (uygulama açıklamaları)
   • Kategori bazlı detaylı analizler
   • Rekabet analizi

3. MAKİNE ÖĞRENMESİ MODELLERİ:
   • Uygulama başarı tahmini
   • Fiyat optimizasyonu
   • Kategori sınıflandırması
   • İndirme sayısı tahmini

4. GÖRSELLEŞTİRME VE RAPORLAMA:
   • İnteraktif dashboard (Streamlit)
   • Otomatik rapor üretimi
   • API entegrasyonu
''')

📝 SONRAKİ ADIMLAR

1. VERİ TEMİZLEME VE İŞLEME:
   • Eksik verilerin doldurulması
   • Aykırı değerlerin tespiti ve işlenmesi
   • Veri tiplerinin düzeltilmesi
   • Feature engineering

2. İLERİ ANALİZLER:
   • Zaman serisi analizi (güncelleme tarihleri)
   • Sentiment analizi (uygulama açıklamaları)
   • Kategori bazlı detaylı analizler
   • Rekabet analizi

3. MAKİNE ÖĞRENMESİ MODELLERİ:
   • Uygulama başarı tahmini
   • Fiyat optimizasyonu
   • Kategori sınıflandırması
   • İndirme sayısı tahmini

4. GÖRSELLEŞTİRME VE RAPORLAMA:
   • İnteraktif dashboard (Streamlit)
   • Otomatik rapor üretimi
   • API entegrasyonu



In [21]:
# Notebook'un çalıştığını test edelim
print("✅ Notebook başarıyla güncellendi!")
print("\n📊 Top Apps Analizi eklendi:")
print("  • Top Free, Paid, Grossing listeleri")
print("  • Kategori dağılım analizleri")
print("  • Çapraz liste karşılaştırmaları")
print("  • Görselleştirmeler ve istatistikler")

✅ Notebook başarıyla güncellendi!

📊 Top Apps Analizi eklendi:
  • Top Free, Paid, Grossing listeleri
  • Kategori dağılım analizleri
  • Çapraz liste karşılaştırmaları
  • Görselleştirmeler ve istatistikler
